In [298]:
# Code to check if left or right mouse buttons were pressed
import win32api
from time import sleep
import pyautogui as mouse
#import tkinter as tk
import tkinter as tk
from tkinter import filedialog
import threading
from datetime import datetime
import keyboard  # using module keyboard
import pythoncom, pyHook
import os 

In [299]:
left_click_text = "Left click "
right_click_text = "Right click "
double_prefix = "Double "
delay_text = "Delay "
double_click_delay = 200
file_extension = ".autoclicker"

In [300]:
record_toggle = 0
play_toggle = 0
repetition_entry = 0
infinite_checkbox = 0

is_shift = False
is_ctrl = False
is_alt = False

is_recording = False
is_running = False

last_click = datetime.now()

repetitions = 0
infinite_repetitions = 0

is_filedialog_open = False

In [301]:
def get_text():
    text_field.configure(state="normal")
    s = text_field.get("1.0", tk.END)
    text_field.configure(state="disabled")    
    return s

In [302]:
def add_text(text):
    text_field.configure(state="normal")
    text_field.insert(tk.END, text + '\n')
    text_field.configure(state="disabled")    

In [303]:
def delete_text():
    text_field.configure(state="normal")
    text_field.delete("1.0", tk.END)
    text_field.configure(state="disabled")    

In [304]:
def get_delay():
    global last_click
    return int((datetime.now() - last_click).total_seconds() * 1000)

In [305]:
def remove_from_end(count):
    s = get_text()
    delete_text()
    add_text(s[:len(s) - count])

In [306]:
def remove_last_line():
    s = text_field.get("1.0", tk.END)
    remove_from_end(len(s) - s.rfind('\n', 0, len(s)-5))

In [307]:
def add_delay():
    global last_click
    delay = get_delay()
    
    add_text(delay_text + str(delay))
        
    last_click = datetime.now()
    return delay

In [308]:
def record():     
    thread = threading.Thread(target=record_thread)
    thread.start()

def record_thread():
    global is_recording
    global last_click
    global play_toggle
    global hook_manager
    
    play_toggle.config(state="disabled")
    
    if is_recording:
        is_recording = False
        return
    is_recording = True
    
    text_field.configure(state="disabled")
    state_left = win32api.GetKeyState(0x01)  # Left button down = 0 or 1. Button up = -127 or -128
    state_right = win32api.GetKeyState(0x02)  # Right button down = 0 or 1. Button up = -127 or -128
    
    last_click = datetime.now()
    
    while is_recording:
        a = win32api.GetKeyState(0x01)
        b = win32api.GetKeyState(0x02)

        if a != state_left or b != state_right:
            pos = mouse.position()
            text = ""
                        
            if a != state_left:  # Button state changed
                state_left = a
                if a < 0:
                    text += left_click_text
                else:
                    continue
                
            if b != state_right:  # Button state changed
                state_right = b
                if b < 0:
                    text += right_click_text
                else:
                    continue
                    
            click_delay = add_delay()
            if click_delay < double_click_delay:
                remove_last_line()
                remove_last_line()
                text = double_prefix + text
                   
            text += str(pos.x) + ", " + str(pos.y)
            
            add_text(text)
            
    remove_last_line()
    remove_last_line()
    
    text_field.configure(state="normal")
    play_toggle.config(state="normal")

In [309]:
def parse(_line):
    line = _line.strip()
    clicks = 1
    if line.startswith(double_prefix):
        clicks = 2
        line = line.replace(double_prefix, "")
    
    click = parse_click(line)
    if click != None:
        mouse.click(button=click[0], clicks=clicks, x=click[1], y=click[2])
        return
    elif line.startswith(delay_text):
        line = line.replace(delay_text, "")
        delay = int(line) / 1000
        sleep(delay)      

In [310]:
def parse_click(line):
    s = line
    if line.startswith(left_click_text):
        s = line[len(left_click_text):len(line)]
        side = "left"
    elif line.startswith(right_click_text):
        s = line[len(right_click_text):len(line)]
        side = "right"
    
    if len(s) != len(line):
        p = s.split(", ")
        return (side, int(p[0]), int(p[1]))
    return None

In [311]:
def play():    
    thread = threading.Thread(target=play_thread)
    thread.start()

def play_thread():
    global is_running
    global text_field
    global record_toggle
    record_toggle.config(state="disabled")
    
    if is_running:
        is_running = False
        return
    is_running = True
    
    lines = get_text().splitlines()
    
    text_field.configure(state="disabled")
        
    repetition_counter = 1
    print(str(infinite_repetitions) + " - " + str(repetition_counter))
    
    while is_running and (infinite_repetitions.get() == 1 or repetition_counter <= repetitions.get()):
        for line in lines:
            if not is_running:
                break
            parse(line)  
        repetition_counter += 1
        
    on_stop(False)
    text_field.configure(state="normal")  
    record_toggle.config(state="normal")

In [312]:
def toggle_button(toggle, action=None):
    if action != None:
        action()
        
    if toggle.config('relief')[-1] == 'sunken':
        toggle.config(relief="raised")
    else:
        toggle.config(relief="sunken")

In [313]:
def create_toggle(text, side, action):
    toggle = tk.Button(top_buttons, text=text, relief="raised", command=lambda: toggle_button(toggle, action))
    toggle.pack(side=side, pady=5, padx=5)
    return toggle

In [314]:
def add_text_field():
    global text_field
    scrollbar = tk.Scrollbar(root)
    text_field = tk.Text(root, height=4)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
    text_field.pack(side=tk.LEFT, fill=tk.BOTH)
    scrollbar.config(command=text_field.yview)
    text_field.config(yscrollcommand=scrollbar.set)

In [315]:
def add_top_buttons():
    global top_buttons
    top_buttons = tk.Text(root, state="disabled", cursor='arrow')    
    top_buttons.pack(side=tk.TOP, fill=tk.BOTH)

In [316]:
def add_int_entry(side):
    int_var = tk.IntVar()
    int_entry = tk.Entry(top_buttons, width=5, textvariable=int_var)
    int_entry.pack(side=side, pady=5, padx=5)
    return int_var, int_entry

In [317]:
def add_checkbox(text, side, action):
    int_var = tk.IntVar()
    int_var.set(0)
    checkbox = tk.Checkbutton(top_buttons, text=text, variable=int_var, command=action)
    checkbox.pack(side=side, pady=5, padx=5)
    return int_var, checkbox

In [318]:
def add_menu():
    # create a toplevel menu
    global menu_bar
    global file_menu
    global edit_menu
    global help_menu
    
    menu_bar = tk.Menu(root)

    # create a pulldown menu, and add it to the menu bar
    file_menu = tk.Menu(menu_bar, tearoff=0)
    file_menu.add_command(label="Open", command=on_open)
    file_menu.add_command(label="Save", command=on_save)
    file_menu.add_separator()
    file_menu.add_command(label="Exit", command=on_closing)
    menu_bar.add_cascade(label="File", menu=file_menu)

    # create more pulldown menus
    edit_menu = tk.Menu(menu_bar, tearoff=0)
    edit_menu.add_command(label="Cut", command=on_cut)
    edit_menu.add_command(label="Copy", command=on_copy)
    edit_menu.add_command(label="Paste", command=on_paste)
    menu_bar.add_cascade(label="Edit", menu=edit_menu)

    help_menu = tk.Menu(menu_bar, tearoff=0)
    help_menu.add_command(label="About", command=on_about)
    menu_bar.add_cascade(label="Help", menu=help_menu)

    # display the menu
    root.config(menu=menu_bar)

In [319]:
def on_open():
    on_stop(False)
    root.withdraw()
    file_path = filedialog.askopenfilename(initialdir=os.getcwd(),title = "Select file",filetypes = (("AutoClicker files","*" + file_extension),))
    root.deiconify()
    print(file_path)
    
    if len(file_path) == 0:
        return False
        
    file = open(file_path, "r")
    add_text(file.read())
    file.close()
    on_stop(False)
    return True

In [320]:
def on_save():
    on_stop(False)
    root.withdraw()
    file_path = filedialog.asksaveasfilename(initialdir=os.getcwd(),title = "Select file",filetypes = (("AutoClicker files","*" + file_extension),))
    root.deiconify()
    print(file_path)
    
    if len(file_path) == 0:
        return False
    
    if not file_path.endswith(file_extension):
        file_path += file_extension
    
    file = open(file_path, "w+")
    file.write(get_text())
    file.close()
    on_stop(False)
    return True

In [321]:
def on_cut():
    pass

In [322]:
def on_copy():
    pass

In [323]:
def on_paste():
    pass

In [324]:
def on_about():
    pass

In [325]:
def on_infinite_check():
    global repetition_entry
    global is_filedialog_open
    
    is_filedialog_open = True
    state = "normal"
    if infinite_repetitions.get() == 1:
        state="disable"
    repetition_entry.config(state=state)
    is_filedialog_open = False

In [326]:
def on_closing():
    global is_running
    global is_recording
    is_recording = False
    is_running = False    
    pythoncom.PumpWaitingMessages()   
    root.destroy()

In [327]:
def on_stop(remove):
    global is_running
    global play_toggle
    global is_recording
    global record_toggle
    global text_field
    
    is_running = False
    is_recording = False
     
    if remove:
        remove_last_line()
        
    for toggle in play_toggle, record_toggle:
        if toggle.config('relief')[-1] == 'sunken':
            toggle.config(relief="raised")
            
    text_field.configure(state="normal")

In [328]:
def on_keyboard_up_event(event):
    global is_shift
    global is_ctrl
    global is_alt
    global is_filedialog_open
    
    if is_filedialog_open:
        return False
    
    if is_recording or is_running: 
        key = str(event.GetKey())
        
        if "control" in key:
            is_ctrl = False
        elif "menu" in key:
            is_alt = False
        elif "shift" in key:
            is_shift = False
            
    return True

In [329]:
def on_keyboard_down_event(event):
    global is_recording
    global text_field
    global is_shift
    global is_ctrl
    global is_alt
    global is_filedialog_open
    
    if is_filedialog_open:
        return False

    if is_recording or is_running:
        
        key = str(event.GetKey())
        
        text = ""
        if is_ctrl:
            text += "strg + "
        if is_alt:
            text += "alt + "
        if is_shift:
            text += "shift + "
        
        
        if "control" in key:
            is_ctrl = True
            return True
        elif "menu" in key:
            is_alt = True
            return True
        elif "shift" in key:
            is_shift = True
            return True
        
        click_delay = add_delay()
        add_text(text + key)
        
        if str(event.GetKey()) == "F7":
            on_stop(True)
    return True

In [330]:
root = tk.Tk()
root.title("Auto Clicker")

add_top_buttons()
add_text_field()
add_menu()

record_toggle = create_toggle("Record", tk.LEFT, record)
play_toggle = create_toggle("Play", tk.LEFT, play)
infinite_repetitions, infinite_checkbox = add_checkbox("Infinite", tk.RIGHT, on_infinite_check)
repetitions, repetition_entry = add_int_entry(tk.RIGHT)

hook_manager = pyHook.HookManager()
hook_manager.KeyDown = on_keyboard_down_event
hook_manager.KeyUp = on_keyboard_up_event
hook_manager.HookKeyboard()
    
root.geometry("500x500")
root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()

PY_VAR18 - 1
PY_VAR18 - 1
PY_VAR18 - 1
PY_VAR18 - 1
PY_VAR18 - 1
PY_VAR18 - 1
